# Cleaning and Analyzing an eBay Car Sales Dataset

In this project we will explore, clean, and analyze a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.

Our purpose here is to demonstrate an ability to clean data and we will therefore focus less on thorough analysis.

A copy of the dataset can be found [here](https://data.world/data-society/used-cars-data). However, this analysis will be completed using a 50,000 row sample that includes modifications for an increased data cleaning challenge.

# Findings 

The top 5 most frequently sold brands are all German car brands, which, considering the source of the dataset, is as we might expect. 

\\$2,000 to $8,000 USD appears to be the sweet spot for the used car price point range.

There are price point categories we can identify fitting low, medium, and high within the \\$2,000 - $8,000 range. Low being around \\$3,000, medium around \\$5,000 and high around \\$8,000. Of the top five brands, one falls in low, Opel, one falls in medium, Volkswagen, and three fall in high-end, BMW, Mercedez Benz, and Audi.

There doesn't appear to be a notable odometer variance among brands.

____
There is a range at which people more frequently sell their cars, 70-80k miles on the odometer. 

____

Vehicle types can be classified into three price groups: high(\~\\$10,000), medium(\~\\$5,000) and low(\~\\$3,000). 

SUV, coupe, and convertible are high price points.  
Van, sedan, and crossover are medium price points.  
Compact and other are low price points.  


There isn't a notable pattern between vehicle type and mean odometer. 


# Initial Data Exploration

In [1]:
#import the relevant python libraries and read our file.
import numpy as np
import pandas as pd

autos = pd.read_csv('autos.csv', encoding = 'Latin-1') # File was found not to be in default UTF-8 encoding 

In [2]:
# quick exploration of the data
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


# Observations of Data and Actions to Take

The summary data tells us that there are a few areas where the data can be improved upon. Below are initial observations.

- The columns ought to be renamed to follow snakecase convention and could be more descriptive.

- Certain columns are in _object_ type and should be changed to either _int_ or _float_ type in preparation for analysis. Relevant columns include: __price__ and __odometer__.

- There are columns with _null_ values and these need to be analyzed in order to make descisions on how to handle them. i.e. do we leave these values alone, find and include the missing information, or completely remove the values/rows/columns. Target columns include: __vehicleType__, __gearbox__, __model__, __fuelType__, and __notRepairedDamage__.

- The __name__ column has information encoded within it that may be worth pulling out into their own columns, such as the model and additional features of the vehicles. Exploration is needed to understand encoded contents.

- There are severeal columns that are in German and should be translated for our English speaking audience. Relevant columns include: __seller__, __offerType__, __vehicleType__, __gearbox__, __fuelType__, and __notRepairedDamage__. Similarly, decisions will be made where relevant to best translate for an American audience, such as changing from km to mile.

- Several columns represent date values but are not in datetime format. Columns: __dateCrawled__, __lastSeen__, __dateCreated__, __monthOfRegistration__, and __yearOfRegistration__


### Renaming Columns

As a first step, we will rename our column names to follow snakecase and be more accurate in their descriptions. There are several ways we can do this, below are two options. 

__Option 1__
~~~python
# assign to series object
autos_columns = autos.columns

# list of new column names
new_column_names = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest', 'vehicle_test', 'registration_year', 'gearbox', 'power_ps', 'model', 'odometer', 'registration_month', 'fuel_type', 'brand', 'unrepaired_damage', 'ad_created', 'count_pictures', 'postal_code', 'last_seen']

# replacing the old with the new
autos.columns = new_column_names
~~~
__Option 2__
~~~python
# change column names for better descriptions and into camel case
autos.rename({'yearOfRegistration': 'registration_year', 'monthOfRegistration': 'registration_month', 'notRepairedDamage': 'unrepaired_damage', 'dateCreated': 'ad_created', 'dateCrawled': 'date_crawled', 'offerType': 'offer_type', 'vehicleType': 'vehicle_type', 'powerPS': 'power_ps', 'fuelType': 'fuel_type', 'nrOfPictures': 'count_pictures', 'postalCode': 'postal_code', 'lastSeen': 'last_seen'}, axis = 1, inplace = True)
~~~

While both of these code snippets accomplish the same thing, we will use <u>option 2</u> because it takes better advantage of pandas syntax and may be more readable relative to the clarity of 'before' and 'after'. 

In [3]:
# the original array of our dataset's column names
print('Original Column Names:', '\n', autos.columns)

# change column names for better descriptions and into camel case
autos.rename({'yearOfRegistration': 'registration_year', 'monthOfRegistration': 'registration_month', 'notRepairedDamage': 'unrepaired_damage', 'dateCreated': 'ad_created', 'dateCrawled': 'date_crawled', 'offerType': 'offer_type', 'vehicleType': 'vehicle_type', 'powerPS': 'power_ps', 'fuelType': 'fuel_type', 'nrOfPictures': 'count_pictures', 'postalCode': 'postal_code', 'lastSeen': 'last_seen'}, axis = 1, inplace = True)

# the modified array of our dataset's column names
print('\n')
print('Renamed Column Names:', '\n', autos.columns)

Original Column Names: 
 Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


Renamed Column Names: 
 Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'count_pictures', 'postal_code',
       'last_seen'],
      dtype='object')


In [4]:
# re-examine the first few columns with their new names
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,count_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


# Data Exploration (...continued)

In [5]:
autos.describe(include = 'all') # 'include = all' gives us both categorical and numeric columns

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,count_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 11:37:04,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


# Observations of Data and Actions to Take (Update: 1)

The summary data tells us that there are a few areas where the data can be improved upon. Below are initial observations.

- ~~The columns ought to be renamed to follow snakecase convention and could be more descriptive.~~

- Certain columns are in _object_ type and should be changed to either _int_ or _float_ type in preparation for analysis. Relevant columns include: __price__ and __odometer__.

- There are columns with _null_ values and these need to be analyzed in order to make descisions on how to handle them. i.e. do we leave these values alone, find and include the missing information, or completely remove the values/rows/columns. Target columns include: __vehicle_type__, __gearbox__, __model__, __fuel_type__, and __unrepaired_damage__.

- The __name__ column has information encoded within it that may be worth pulling out into their own columns, such as the model and additional features of the vehicles. Exploration is needed to understand encoded contents.

- There are severeal columns that are in German and should be translated for our English speaking audience. Relevant columns include: __seller__, __offer_type__, __vehicle_type__, __gearbox__, __fuel_type__, and __unrepaired_damage__. Similarly, decisions will be made where relevant to best translate for an American audience, such as changing from km to mile.

- Several columns represent date values but are not in datetime format. Columns: __date_crawled__, __last_seen__, __ad_created__, __registration_month__, and __registration_year__

___

- The __date_crawled__ and __last_seen__ columns may not provide useful information.

- The __count_picutures__ column does not appear to contain any useful information as all values are zero.

- The __registration_year__ appears to have some dirty data, with a min of 1000 and max of 9999.

- The __registration_month__ has a numerical month and has 0 for its minimum and 12 for its maximum. This would leave us with 13 months. We should explore whether there are some miscategorized zero values that should begin with 1. 

- __power_ps__ is the European equivalent of horsepower. 1 PS or metric horsepower(EU) is equivalent to .989632 HP or mechanical horsepower(US). For reference 1,000 HP is attainable, but very rare (and expensive). The maximum we see in our __power_ps__ column of 17,700 is therefore unreasonable. This column should be invesitgated for any cars over 1,000 HP. The conversion formula we will use is HP = (PS / 1.104).

- Check whether __odometer__ values are ranges, and if they are clean the data to make it more clear.

- Several columns are only one of two values. Their value towards analysis should be considered. Relevant columns include: __seller__, __offer_type__, __abtest__, __gearbox__, and __unrepaired_damage__

### Changing <u>Price</u> and <u>Odometer</u> Column Values to Numbers

In [6]:
#change values in price column
autos['price'] = (autos['price'] #select price column
                          .str.replace('$','') #remove the '$' string value
                          .str.replace(',','') #remove the ',' string value
                          .astype(float) # change to float
                 )

In [7]:
print(autos.iloc[:5,4]) #print the first five columns of price
print('\n')
print('Max Price: ', autos['price'].max()) # check the max price to validate data

0    5000.0
1    8500.0
2    8990.0
3    4350.0
4    1350.0
Name: price, dtype: float64


Max Price:  99999999.0


From the above we see that there may be outliers in the __price__ column, below we explore that further and see that there are both low and high outliers that will need to be cleaned.

In [8]:
# Top 5 lowest prices
print(autos['price']
              .value_counts() # Create a frequency table of prices
              .sort_index(ascending=True) # Sort from lowest price to highest
              .head()) # Filter only the top results

0.0    1421
1.0     156
2.0       3
3.0       1
5.0       2
Name: price, dtype: int64


In [9]:
# Top 5 highest prices
print(autos['price']
              .value_counts() # Create a frequency table of prices
              .sort_index(ascending = False) # Sort from highest price to highest
              .head(15)) # Filter only the top results

99999999.0    1
27322222.0    1
12345678.0    3
11111111.0    2
10000000.0    1
3890000.0     1
1300000.0     1
1234566.0     1
999999.0      2
999990.0      1
350000.0      1
345000.0      1
299000.0      1
295000.0      1
265000.0      1
Name: price, dtype: int64


___
Now we evaluate __odometer__.

In [10]:
autos['odometer'] = (autos['odometer'] # select the odomoter column as a series
                     .str.replace('km','') # replace km with an empty string
                     .str.replace(',','') # replace the comma with an empty string
                     .astype(int)) # change the datatype from string to integer 

In [11]:
print(autos.iloc[:5,11]) #print the first fove columns of odometer
print('\n')
print('Max Odomoter: ', autos['odometer'].max()) # check the max price to validate data
print('\n')
print(autos['odometer'].value_counts()) # Create a frequency table of the odometer values

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer, dtype: int64


Max Odomoter:  150000


150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: odometer, dtype: int64


The below changes the name of __odometer__ to __odometer_km__ because the data is no longer self explanatory.

In [12]:
autos.rename({'odometer': 'odometer_km'}, axis = 1, inplace = 1) # Change the title of column odometer to odometer_km

print(autos.columns) # check to confirm change

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'count_pictures', 'postal_code',
       'last_seen'],
      dtype='object')


___
The values within __price__ and __odometer__ have now been changed to integers and floats, and the __odometer__ column has been renamed to __odometer_km__. 

During this process we learned there is bad pricing data that will need to be cleaned, and the odometer readings fall within ranges and are not unique entries. We will now take steps to clean these columns. Because there is a benefit to having ranges in numerical format, we will leave the odometer ranges as is for now.
___

In [13]:
expensive_makes_and_models_bool = autos['price'] >= 150000

#column_explore = ['brand', 'model']

expensive_makes_and_models = autos.loc[expensive_makes_and_models_bool,['price', 'model', 'brand','registration_year']].value_counts().sort_index(ascending=False).head(15)
print(expensive_makes_and_models)

price       model        brand          registration_year
99999999.0  s_klasse     mercedes_benz  1999                 1
27322222.0  c4           citroen        2014                 1
12345678.0  vectra       opel           2001                 1
            v40          volvo          2018                 1
            punto        fiat           2017                 1
11111111.0  escort       ford           1973                 1
999999.0    transporter  volkswagen     1981                 1
            focus        ford           2009                 1
999990.0    jetta        volkswagen     1985                 1
350000.0    911          porsche        2016                 1
299000.0    911          porsche        1977                 1
295000.0    911          porsche        2015                 1
265000.0    911          porsche        2016                 1
259000.0    z_reihe      bmw            2001                 1
250000.0    x_reihe      bmw            2015                

Looking at the above and completing a little research on cars, we can see that cars above 350,000 start to reach unreasonable prices for the model and brand of the cars for sale. 

In [14]:
# remove values below 99 and above 900,000
autos = autos[autos["price"].between(99,900000)] 

In [15]:
expensive_makes_and_models_bool = autos['price'] >= 150000

#column_explore = ['brand', 'model']

expensive_makes_and_models = autos.loc[expensive_makes_and_models_bool,['price', 'model', 'brand','registration_year']].value_counts().sort_index(ascending=False).head(15)
print(expensive_makes_and_models)

price     model     brand          registration_year
350000.0  911       porsche        2016                 1
299000.0  911       porsche        1977                 1
295000.0  911       porsche        2015                 1
265000.0  911       porsche        2016                 1
259000.0  z_reihe   bmw            2001                 1
250000.0  x_reihe   bmw            2015                 1
220000.0  911       porsche        2008                 1
198000.0  911       porsche        2015                 1
197000.0  911       porsche        1998                 1
190000.0  z_reihe   bmw            2000                 1
180000.0  andere    mercedes_benz  2012                 1
175000.0  andere    audi           2016                 1
169999.0  g_klasse  mercedes_benz  2012                 1
169000.0  911       porsche        2013                 1
163991.0  911       porsche        2014                 1
dtype: int64


Above we removed prices above 900,000 and below 99. 

The rationale behind this being that we are not too interested in outliers. There are plenty of ads for $0, and that can be considered an ad with an 'unreal' offer. Cars can sell for cheap, however, and the lower limit was set to 100, which is where distribution starts to pick up. 

A similar rationale can be taken for the higher priced cars. Cars can get quite expensive, but there is a point where that becomes unreasonable. After 350,000 things price shoots up fast and the model and brand don't justify the price. 900,000 was used because that was the next highest value after the 350,000 porsche 911, which was a brand new car for the time this data was scraped (2016).

# Observations of Data and Actions to Take (Update: 2)

The summary data tells us that there are a few areas where the data can be improved upon. Below are initial observations.

- ~~The columns ought to be renamed to follow snakecase convention and could be more descriptive.~~

- ~~Certain columns are in _object_ type and should be changed to either _int_ or _float_ type in preparation for analysis. Relevant columns include: __price__ and __odometer__.~~

- There are columns with _null_ values and these need to be analyzed in order to make descisions on how to handle them. i.e. do we leave these values alone, find and include the missing information, or completely remove the values/rows/columns. Target columns include: __vehicle_type__, __gearbox__, __model__, __fuel_type__, and __unrepaired_damage__.

- The __name__ column has information encoded within it that may be worth pulling out into their own columns, such as the model and additional features of the vehicles. Exploration is needed to understand encoded contents.

- There are severeal columns that are in German and should be translated for our English speaking audience. Relevant columns include: __seller__, __offer_type__, __vehicle_type__, __gearbox__, __fuel_type__, and __unrepaired_damage__. Similarly, decisions will be made where relevant to best translate for an American audience, such as changing from km to mile.

- Several columns represent date values but are not in datetime format. Columns: __date_crawled__, __last_seen__, __ad_created__, __registration_month__, and __registration_year__


___

- The __date_crawled__ and __last_seen__ columns may not provide useful information.

- The __count_picutures__ column does not appear to contain any useful information as all values are zero.

- The __registration_year__ appears to have some dirty data, with a min of 1000 and max of 9999.

- The __registration_month__ has a numerical month and has 0 for its minimum and 12 for its maximum. This would leave us with 13 months. We should explore whether there are some miscategorized zero values that should begin with 1. 

- __power_ps__ is the European equivalent of horsepower. 1 PS or metric horsepower(EU) is equivalent to .989632 HP or mechanical horsepower(US). For reference 1,000 HP is attainable, but very rare (and expensive). The maximum we see in our __power_ps__ column of 17,700 is therefore unreasonable. This column should be invesitgated for any cars over 1,000 HP. The conversion formula we will use is HP = (PS / 1.104).

- ~~Check whether __odometer__ values are ranges, and if they are clean the data to make it more clear.~~

- Several columns are only one of two values. Their value towards analysis should be considered. Relevant columns include: __seller__, __offer_type__, __abtest__, __gearbox__, and __unrepaired_damage__

___


- ~~Evaluate the extent of bad __pricing__ data and determine the best way to clean the series.~~

### Date Range Formatting

There are several columns that represent dates and times, but are not in numerical or datetime format. 

Let's first look at those that are still in object format. __date_crawled__, __last_seen__, and __ad_created__

In [16]:
# print the first five rows of the date_crawled, last_seen, and ad_created columns.
print(autos[['date_crawled', 'last_seen', 'ad_created']][:5])

          date_crawled            last_seen           ad_created
0  2016-03-26 17:47:46  2016-04-06 06:45:54  2016-03-26 00:00:00
1  2016-04-04 13:38:56  2016-04-06 14:45:08  2016-04-04 00:00:00
2  2016-03-26 18:57:24  2016-04-06 20:15:37  2016-03-26 00:00:00
3  2016-03-12 16:58:10  2016-03-15 03:16:28  2016-03-12 00:00:00
4  2016-04-01 14:38:50  2016-04-01 14:38:50  2016-04-01 00:00:00


We can see that these are uniformly formatted in ((year, month, day) (hour, minute, seconds))

In [17]:
date_crawled_noclock = autos['date_crawled'].str[:10] # Create a series that saves only the first 10 string values (year-month-day)

print(date_crawled_noclock[:10])

0    2016-03-26
1    2016-04-04
2    2016-03-26
3    2016-03-12
4    2016-04-01
5    2016-03-21
6    2016-03-20
7    2016-03-16
8    2016-03-22
9    2016-03-16
Name: date_crawled, dtype: object


In [18]:
date_crawled_noclock.value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2016-03-05    0.025372
2016-03-06    0.014033
2016-03-07    0.036047
2016-03-08    0.033207
2016-03-09    0.033000
2016-03-10    0.032274
2016-03-11    0.032606
2016-03-12    0.036897
2016-03-13    0.015671
2016-03-14    0.036648
2016-03-15    0.034305
2016-03-16    0.029455
2016-03-17    0.031528
2016-03-18    0.012893
2016-03-19    0.034762
2016-03-20    0.037788
2016-03-21    0.037228
2016-03-22    0.032917
2016-03-23    0.032295
2016-03-24    0.029455
2016-03-25    0.031507
2016-03-26    0.032316
2016-03-27    0.031113
2016-03-28    0.034948
2016-03-29    0.034098
2016-03-30    0.033725
2016-03-31    0.031860
2016-04-01    0.033704
2016-04-02    0.035591
2016-04-03    0.038596
2016-04-04    0.036544
2016-04-05    0.013059
2016-04-06    0.003171
2016-04-07    0.001389
Name: date_crawled, dtype: float64

__date_crawled__ will provide little value to our analysis apart from context. We can see that the website was crawled over a month's time and have fairly even distribution.

In [19]:
last_seen_noclock = autos['last_seen'].str[:10] # Create a series that saves only the first 10 string values (year-month-day)

print(last_seen_noclock[:10])

0    2016-04-06
1    2016-04-06
2    2016-04-06
3    2016-03-15
4    2016-04-01
5    2016-04-06
6    2016-03-23
7    2016-04-07
8    2016-03-26
9    2016-04-06
Name: last_seen, dtype: object


In [20]:
last_seen_noclock.value_counts(normalize = True, dropna = False).sort_index(ascending = True)

2016-03-05    0.001078
2016-03-06    0.004312
2016-03-07    0.005431
2016-03-08    0.007317
2016-03-09    0.009597
2016-03-10    0.010634
2016-03-11    0.012396
2016-03-12    0.023775
2016-03-13    0.008872
2016-03-14    0.012624
2016-03-15    0.015878
2016-03-16    0.016438
2016-03-17    0.028087
2016-03-18    0.007317
2016-03-19    0.015795
2016-03-20    0.020666
2016-03-21    0.020563
2016-03-22    0.021350
2016-03-23    0.018593
2016-03-24    0.019775
2016-03-25    0.019132
2016-03-26    0.016666
2016-03-27    0.015546
2016-03-28    0.020853
2016-03-29    0.022283
2016-03-30    0.024708
2016-03-31    0.023817
2016-04-01    0.022843
2016-04-02    0.024895
2016-04-03    0.025123
2016-04-04    0.024542
2016-04-05    0.125054
2016-04-06    0.221939
2016-04-07    0.132102
Name: last_seen, dtype: float64

__last_seen__ similarly provides context, but little analytical value. More recent dates have a higher distribution, which makes sense because this is a value that is updated with each crawl.

In [21]:
ad_created_noclock = autos['ad_created'].str[:10] # Create a series that saves only the first 10 string values (year-month-day)

print(ad_created_noclock[:10])

0    2016-03-26
1    2016-04-04
2    2016-03-26
3    2016-03-12
4    2016-04-01
5    2016-03-21
6    2016-03-20
7    2016-03-16
8    2016-03-22
9    2016-03-16
Name: ad_created, dtype: object


In [22]:
print(ad_created_noclock.value_counts(normalize = True, dropna = False).sort_index(ascending = True).head(5))
ad_created_noclock.value_counts(normalize = True, dropna = False).sort_index(ascending = True).tail(20)

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
Name: ad_created, dtype: float64


2016-03-19    0.033642
2016-03-20    0.037850
2016-03-21    0.037456
2016-03-22    0.032730
2016-03-23    0.032129
2016-03-24    0.029393
2016-03-25    0.031632
2016-03-26    0.032378
2016-03-27    0.031030
2016-03-28    0.035052
2016-03-29    0.034057
2016-03-30    0.033539
2016-03-31    0.031901
2016-04-01    0.033684
2016-04-02    0.035280
2016-04-03    0.038845
2016-04-04    0.036897
2016-04-05    0.011794
2016-04-06    0.003254
2016-04-07    0.001244
Name: ad_created, dtype: float64

__ad_created__ adds context, but little analytical value. The dates go back to 2015, but there is a higher distribution towards recency. There could be potential for looking into makes/models that don't sell, but there are missing datapoints that would make this viable, such as completed sale history.

___

The two remaining columns that have dates we should evaluate are __registration_year__ and __registration_month__. As noted earlier:

- The __registration_year__ appears to have some dirty data, with a min of 1000 and max of 9999.

- The __registration_month__ has a numerical month and has 0 for its minimum and 12 for its maximum. This would leave us with 13 months. We should explore whether there are some miscategorized zero values that should begin with 1. 


In [23]:
# Look at the last values' distribution in our registration_year series
print(autos['registration_year'].value_counts().sort_index(ascending = False).tail(30))

1964    12
1963     8
1962     4
1961     6
1960    22
1959     6
1958     4
1957     2
1956     4
1955     2
1954     2
1953     1
1952     1
1951     2
1950     1
1948     1
1943     1
1941     2
1939     1
1938     1
1937     4
1934     2
1931     1
1929     1
1927     1
1910     2
1800     2
1111     1
1001     1
1000     1
Name: registration_year, dtype: int64


We know the dataset must have a maximum end year of 2016, as the data was pulled in May of 2016. 

The minimum will require more discretion - cars became popularized in 1908, but the likelihood that these cars are being sold are at least unimportant for the current analysis. Therefore, we will start our count in 1959, right before a popular year with a long sequence of low volume years leading up to it.

In [24]:
# remove rows with years out of range
autos = autos[autos['registration_year'].between(1959,2016)]

In [25]:
# confirmation our changes were made
print(autos['registration_year'].value_counts().sort_index(ascending = False).tail())

1963     8
1962     4
1961     6
1960    22
1959     6
Name: registration_year, dtype: int64


In [26]:
autos['registration_year'].value_counts(normalize=True).sort_index(ascending=False)

2016    0.025964
2015    0.008223
2014    0.014288
2013    0.017288
2012    0.028251
2011    0.034920
2010    0.034251
2009    0.044891
2008    0.047697
2007    0.049057
2006    0.057582
2005    0.062848
2004    0.058251
2003    0.058143
2002    0.053460
2001    0.056740
2000    0.067013
1999    0.062136
1998    0.050589
1997    0.041632
1996    0.029287
1995    0.025769
1994    0.013532
1993    0.009065
1992    0.007921
1991    0.007295
1990    0.007165
1989    0.003691
1988    0.002870
1987    0.001554
1986    0.001532
1985    0.002007
1984    0.001101
1983    0.001101
1982    0.000885
1981    0.000583
1980    0.001748
1979    0.000734
1978    0.000906
1977    0.000475
1976    0.000453
1975    0.000388
1974    0.000518
1973    0.000496
1972    0.000712
1971    0.000561
1970    0.000799
1969    0.000410
1968    0.000561
1967    0.000561
1966    0.000475
1965    0.000367
1964    0.000259
1963    0.000173
1962    0.000086
1961    0.000129
1960    0.000475
1959    0.000129
Name: registra

While there are some interesting years (1980) and (2015), the distribution is as expected. There are more cars on the road, and therefore for sale, in years that are have more recency.

___

Below we see the distribution for __registration_months__ and that there is data that doesn't quite make sense. There are 13 months when we would expect two, indicating there is something wrong with how these values are categorized. Given the fairly even distribution here, we can't assume that those values categorized as '0' should have been in '1' or January. One hypothesis is that '0' months are uncategorized or null. 

In [27]:
# create a frequency table of the registration month column - here we converted it to percentages using 'normalize'
autos['registration_month'].value_counts(normalize=True, dropna = False).sort_index()

0     0.083330
1     0.066409
2     0.061316
3     0.104006
4     0.083697
5     0.083351
6     0.088466
7     0.079898
8     0.064963
9     0.069733
10    0.075193
11    0.069560
12    0.070078
Name: registration_month, dtype: float64

To clean the data we will categorize 0 as Nan and all other values equal to their descriptive numerical counterparts. e.g. 1 = January and 12 = December

In [28]:
#Create a dictionary with a key value relationship of month number and descriptive month
month_dic = {1: 'January', 2: 'February', 3:'March', 4: 'April', 
             5: 'May', 6: 'June', 7: 'July', 8: 'August', 
             9: 'September', 10: 'October', 11: 'November', 12: 'December', 0: np.nan}

# use the dictionary to map the descriptive months to the numerical key in the registration_month column
autos['registration_month'] = autos['registration_month'].map(month_dic)

In [29]:
# Check that the values have changed by running the same code as before
autos['registration_month'].value_counts(normalize=True, dropna = False).sort_index()

April        0.083697
August       0.064963
December     0.070078
February     0.061316
January      0.066409
July         0.079898
June         0.088466
March        0.104006
May          0.083351
November     0.069560
October      0.075193
September    0.069733
NaN          0.083330
Name: registration_month, dtype: float64

The remapping of our value keys was successful, however they are now out of order. We will recreate that order below.

In [30]:
# Create a list that matches exactly the values we want to sort and in their proper order
months = ["January", 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
# Use pandas categorical to create an ordered relationship for the months list that will be applied to the series
autos['registration_month'] = pd.Categorical(autos['registration_month'], categories = months, ordered = True)

In [31]:
# Check that values have changed and drop the NaN values
autos['registration_month'].value_counts(normalize=True, dropna = True).sort_index()

January      0.072446
February     0.066890
March        0.113460
April        0.091305
May          0.090928
June         0.096508
July         0.087161
August       0.070869
September    0.076072
October      0.082029
November     0.075884
December     0.076449
Name: registration_month, dtype: float64

__registration_months__ tells us the month in which a car was first registered, something that happens when a new car is purchased. We can assume that there is a correlation between __registration_months__ and <u>new car</u> purchase behavior. 

There is an uptick in car registrations in the early-mid months of the year, March to July. This may be a seasonal behavior where people have an increased tendency to buy cars in the Spring and Summer months. Insight will be passed to marketing for further investigation.

# Observations of Data and Actions to Take (Update: 3)

The summary data tells us that there are a few areas where the data can be improved upon. Below are initial observations.

- ~~The columns ought to be renamed to follow snakecase convention and could be more descriptive.~~

- ~~Certain columns are in _object_ type and should be changed to either _int_ or _float_ type in preparation for analysis. Relevant columns include: __price__ and __odometer__.~~

- There are columns with _null_ values and these need to be analyzed in order to make descisions on how to handle them. i.e. do we leave these values alone, find and include the missing information, or completely remove the values/rows/columns. Target columns include: __vehicle_type__, __gearbox__, __model__, __fuel_type__, and __unrepaired_damage__.

- The __name__ column has information encoded within it that may be worth pulling out into their own columns, such as the model and additional features of the vehicles. Exploration is needed to understand encoded contents.

- There are severeal columns that are in German and should be translated for our English speaking audience. Relevant columns include: __seller__, __offer_type__, __vehicle_type__, __gearbox__, __fuel_type__, and __unrepaired_damage__. Similarly, decisions will be made where relevant to best translate for an American audience, such as changing from km to mile.

- ~~Several columns represent date values but are not in datetime format. Columns: __date_crawled__, __last_seen__, __ad_created__, __registration_month__, and __registration_year__~~


___

- ~~The __date_crawled__ and __last_seen__ columns may not provide useful information.~~

- The __count_picutures__ column does not appear to contain any useful information as all values are zero.

- ~~The __registration_year__ appears to have some dirty data, with a min of 1000 and max of 9999.~~

- ~~The __registration_month__ has a numerical month and has 0 for its minimum and 12 for its maximum. This would leave us with 13 months. We should explore whether there are some miscategorized zero values that should begin with 1. ~~

- __power_ps__ is the European equivalent of horsepower. 1 PS or metric horsepower(EU) is equivalent to .989632 HP or mechanical horsepower(US). For reference 1,000 HP is attainable, but very rare (and expensive). The maximum we see in our __power_ps__ column of 17,700 is therefore unreasonable. This column should be invesitgated for any cars over 1,000 HP. The conversion formula we will use is HP = (PS / 1.104).

- ~~Check whether __odometer__ values are ranges, and if they are clean the data to make it more clear.~~

- Several columns are only one of two values. Their value towards analysis should be considered. Relevant columns include: __seller__, __offer_type__, __abtest__, __gearbox__, and __unrepaired_damage__

___


- ~~Evaluate the extent of bad __pricing__ data and determine the best way to clean the series.~~

- Do certain car __brands__ or __models__ have increased __registrations_month__ than other car brands? (There is a loose link between registration and purchase)

### Americanizing the Dataset

The data was pulled from a German website and is a sample of the used German car market. However, our audience is assumed to be American, and so data should be changed to make it more understandable.

We'll start by cleaning the column of bad data and changing the __power_ps__ column to horsepower.

In [32]:
# View dataset in PS
autos['power_ps'][:10]

0    158
1    286
2    102
3     71
4      0
5    150
6     90
7     90
8      0
9     90
Name: power_ps, dtype: int64

In [33]:
# all rows from PS to HP
autos['power_ps'] = autos['power_ps'] / 1.104
# round answers for readability (assuming we won't be doing scientific analysis)
autos['power_ps'] = autos['power_ps'].round(2)

In [34]:
# view dataset in HP
autos['power_ps'][:10]

0    143.12
1    259.06
2     92.39
3     64.31
4      0.00
5    135.87
6     81.52
7     81.52
8      0.00
9     81.52
Name: power_ps, dtype: float64

In [35]:
# rename columns to horse_power
autos.rename({'power_ps':'horse_power'}, axis = 1, inplace = True)

In [36]:
autos[:1]

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,horse_power,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,count_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000.0,control,bus,2004,manuell,143.12,andere,150000,March,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54


We've now converted the values into HP and renamed them. 

Now let's examine the unrealistic HP values.

In [37]:
# boolean that targets HP above 1000
high_hp_bool = autos['horse_power'] > 1000

In [38]:
# create an dataframe object that 
high_hp = autos.loc[high_hp_bool,['horse_power','model','brand','price']].value_counts().sort_index().head(10)
print(high_hp)

horse_power  model     brand          price 
1088.77      scenic    renault        1600.0    1
1177.54      kaefer    volkswagen     3950.0    1
1238.22      3_reihe   peugeot        3500.0    1
1266.30      corolla   toyota         3200.0    1
1268.12      a_klasse  mercedes_benz  750.0     1
             touran    volkswagen     9900.0    1
1272.64      passat    volkswagen     5400.0    1
1543.48      e_klasse  mercedes_benz  2200.0    1
1587.86      focus     ford           600.0     1
1604.17      e_klasse  mercedes_benz  9500.0    1
dtype: int64


In [39]:
# boolean that targets HP below 50
low_hp_bool = autos['horse_power'] < 50

In [40]:
low_hp = autos.loc[low_hp_bool,['horse_power','model','brand','price']].value_counts().sort_index().head(10)
print(low_hp)

horse_power  model  brand       price 
0.0          100    audi        150.0     1
                                450.0     1
                                499.0     1
                                1984.0    1
             145    alfa_romeo  200.0     1
                                500.0     1
             147    alfa_romeo  550.0     1
                                850.0     1
                                1000.0    1
                                1200.0    1
dtype: int64


There are values of horse power that are unrealistic for the car make and model they're attached to. Due to time constraints, we will change values above 1,000 HP and below 50 (the lowest HP model of car in production today is 66) to NaN values rather than finding the outliers.

In [41]:
# combine the bools of the undesirable values we want to remove
new_null_hp_bool = low_hp_bool | high_hp_bool

In [42]:
# replace the HP of unrealistic values into NaN
autos.loc[new_null_hp_bool, 'horse_power'] = np.nan

In [43]:
# check that work is complete
autos['horse_power'].value_counts(dropna = False).sort_index()

50.72       36
51.63       22
52.54      479
53.44       29
54.35     2010
          ... 
956.52       1
980.07       1
987.32       1
999.09       1
NaN       6661
Name: horse_power, Length: 358, dtype: int64

### Americanizing the Dataset (part 2)

Next we will work towards translating key values from German to English. First, let's identify what those are from the columns we identified earlier. 

Relevant columns: __seller__, __offer_type__, __vehicle_type__, __gearbox__, __fuel_type__, and __unrepaired_damage__

___
#### Translating seller Column

In [44]:
# print the unique values in the seller column
print(autos['seller'].unique())

['privat' 'gewerblich']


Translating these into English, 'privat' represents a private seller, while 'gewerblich' represents a commercial seller. Let's replace these with 'private' and 'dealership' respectively. 

In [45]:
# replace seller column German values with English ones
autos.loc[autos['seller'] == 'privat','seller'] = 'private'
autos.loc[autos['seller'] == 'gewerblich','seller'] = 'dealership'
# check that the values have changed
print(autos['seller'].unique())

['private' 'dealership']


___
#### Translating offer_type Column

In [46]:
print(autos['offer_type'].unique())

['Angebot']


There is only one value that translates to offer. This offers no analytical value and we will remove the column.

In [47]:
# delete the offer_type column
del autos['offer_type']
#print columns to confirm 
print(autos.columns)

Index(['date_crawled', 'name', 'seller', 'price', 'abtest', 'vehicle_type',
       'registration_year', 'gearbox', 'horse_power', 'model', 'odometer_km',
       'registration_month', 'fuel_type', 'brand', 'unrepaired_damage',
       'ad_created', 'count_pictures', 'postal_code', 'last_seen'],
      dtype='object')


___
#### Translating vehicle_type Column

In [48]:
print(autos['vehicle_type'].unique())

['bus' 'limousine' 'kleinwagen' 'kombi' nan 'coupe' 'suv' 'cabrio'
 'andere']


Note: translations were not always apparent. Searching car models falling within the vehicle types and using discretion is how this dictionary was defined.

bus: van  
limousine: sedan  
kleinwagen: compact  
kombi: crossover  
coupe: coupe  
suv: suv  
cabrio: convertible  
andere: other  

In [49]:
# create a dictionary that creates a english value with a german key
vt_en = {'bus': 'van', 'limousine': 'sedan', 'kleinwagen': 'compact', 'kombi':'crossover', 'coupe': 'coupe', 'suv':'suv', 'cabrio': 'convertible','andere':'other'}
# 
autos['vehicle_type'] = autos['vehicle_type'].map(vt_en)

In [50]:
print(autos['vehicle_type'].unique())

['van' 'sedan' 'compact' 'crossover' nan 'coupe' 'suv' 'convertible'
 'other']


The values were successfully swapped with English values. We will opt not to classify the other column with the NaN values because there may be analytical value in why these are classified this way.

___
#### Translating gearbox Column

In [51]:
print(autos['gearbox'].unique())

['manuell' 'automatik' nan]


We will replace 'manuell' with 'manual' and 'automatik' with automatic.

In [52]:
autos.loc[autos['gearbox'] == 'manuell', 'gearbox'] = 'manual'
autos.loc[autos['gearbox'] == 'automatik', 'gearbox'] = 'automatic'

print(autos['gearbox'].unique())

['manual' 'automatic' nan]


___
#### Translating fuel_type Column

In [53]:
print(autos['fuel_type'].unique())

['lpg' 'benzin' 'diesel' nan 'cng' 'hybrid' 'elektro' 'andere']


Below we will place the translations directly into our working dictionary. 

In [54]:
#dictionary with our German fuel types translated
ft_en = {'lpg': 'propane', 'benzin': 'gasoline', 'diesel': 'diesel', 'cng': 'compressed natural gas', 'hybrid': 'hybrid', 'elektro':'electric','andere':'other'}

# map the english translations over the german ones
autos['fuel_type'] = autos['fuel_type'].map(ft_en)

# check the work
print(autos['fuel_type'].unique())

['propane' 'gasoline' 'diesel' nan 'compressed natural gas' 'hybrid'
 'electric' 'other']


___
#### Translating unrepaired_damage Column

In [55]:
print(autos['unrepaired_damage'].unique())

['nein' nan 'ja']


We will replace 'nein' with no and 'ja' with yes. 

In [56]:
autos.loc[autos['unrepaired_damage'] == 'nein', 'unrepaired_damage'] = 'no'
autos.loc[autos['unrepaired_damage'] == 'ja', 'unrepaired_damage'] = 'yes'

print(autos['unrepaired_damage'].unique())

['no' nan 'yes']


___

In [57]:
autos

,date_crawled,name,seller,price,abtest,vehicle_type,registration_year,gearbox,horse_power,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,count_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,private,5000.0,control,van,2004,manual,143.12,andere,150000,March,propane,peugeot,no,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,private,8500.0,control,sedan,1997,automatic,259.06,7er,150000,June,gasoline,bmw,no,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,private,8990.0,test,sedan,2009,manual,92.39,golf,70000,July,gasoline,volkswagen,no,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,private,4350.0,control,compact,2007,automatic,64.31,fortwo,70000,June,gasoline,smart,no,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,private,1350.0,test,crossover,2003,manual,NaN,focus,150000,July,gasoline,ford,no,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2016-03-27 14:38:19,Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon,private,24900.0,control,sedan,2011,automatic,216.49,q5,100000,January,diesel,audi,no,2016-03-27 00:00:00,0,82131,2016-04-01 13:47:40
49996,2016-03-28 10:50:25,Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...,private,1980.0,control,convertible,1996,manual,67.93,astra,150000,May,gasoline,opel,no,2016-03-28 00:00:00,0,44807,2016-04-02 14:18:02
49997,2016-04-02 14:44:48,Fiat_500_C_1.2_Dualogic_Lounge,private,13200.0,test,convertible,2014,automatic,62.50,500,5000,November,gasoline,fiat,no,2016-04-02 00:00:00,0,73430,2016-04-04 11:47:27
49998,2016-03-08 19:25:42,Audi_A3_2.0_TDI_Sportback_Ambition,private,22900.0,control,crossover,2013,manual,135.87,a3,40000,November,diesel,audi,no,2016-03-08 00:00:00,0,35683,2016-04-05 16:45:07


The above shows that the changes were successful. However, two columns were missed in the earlier identification, __odometer_km__ and __price__. Those will convert those columns next.

#### Converting km to mph

The __odometer_km__ values are in ranges, and not exact value form. Therefore, to keep this clean formatting there will be an expense of not having exact 1:1 match in our column conversions. To translate kilometer to mph, we divide km by 1.609. This, not giving divisible by 100 values, will require rounding up our mph values to the nearest thousandth.

In [58]:
# Showing our values in sorted form
print(autos['odometer_km'].sort_values().unique())

[  5000  10000  20000  30000  40000  50000  60000  70000  80000  90000
 100000 125000 150000]


In [59]:
#create an empty list for our converted values
odom_vals_mph = []

# assign the unique valeus to an object
odom_vals = autos['odometer_km'].sort_values().unique()

# for loop that converts km to mph and adds it to our list
for km_val in odom_vals:
    mph = km_val / 1.609 # conversion function
    mph = int(round(mph, -3)) # round to nearest 1000
    odom_vals_mph.append(mph) # add to dataset
    
print(odom_vals_mph)

[3000, 6000, 12000, 19000, 25000, 31000, 37000, 44000, 50000, 56000, 62000, 78000, 93000]


We have a list of conversion values, now let's create a dictionary that we will use to replace the values in our series.

In [60]:
# empty dictionary where our final key: values will live
km_mph_dic = {} 

# copy of our mph values, which will be slowly diminished to nothing  
cp_mph_vals = odom_vals_mph

# loop we will use to accomplish associating the key: value pairs
for key in odom_vals:
    for value in cp_mph_vals: # nested for loop method
        km_mph_dic[key] = value # creation of the key: value relationship in our dictionary
        cp_mph_vals.remove(value) # removing the used value to prevent it's use in the next iteration
        break # breaking the current loop so that we will continue to the next without continuing to write in new values
        
        
print(km_mph_dic)

{5000: 3000, 10000: 6000, 20000: 12000, 30000: 19000, 40000: 25000, 50000: 31000, 60000: 37000, 70000: 44000, 80000: 50000, 90000: 56000, 100000: 62000, 125000: 78000, 150000: 93000}


Next we will use our dictionary to remap our __odomoter_km__ values.

In [61]:
# map our mph values over the mph values for the odometer_km values in the series
autos['odometer_km'] = autos['odometer_km'].map(km_mph_dic)

In [62]:
#check the work
print(autos['odometer_km'].sort_values().unique())

[ 3000  6000 12000 19000 25000 31000 37000 44000 50000 56000 62000 78000
 93000]


In [63]:
# rename odometer_km to odometer_miles
autos.rename({'odometer_km': 'odometer_miles'}, axis=1,inplace=True)

In [64]:
#check that column was renamed
autos.columns

Index(['date_crawled', 'name', 'seller', 'price', 'abtest', 'vehicle_type',
       'registration_year', 'gearbox', 'horse_power', 'model',
       'odometer_miles', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'count_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

We have successfully cleaned our odometer column.
___

#### Translating Euro to USD

This data was pulled in 2016, we will use a Euro to USD exchange rate of 1.11, meaning we will divide our Euro numbers by 1.11 to get the time equivalent for U.S. currency. The exchange rate was taken as the average exchange rate for the year, and can be seen [here](https://www.macrotrends.net/2548/euro-dollar-exchange-rate-historical-chart).

In [65]:
# First two values in the column as a reference point
autos['price'][0:2]

0    5000.0
1    8500.0
Name: price, dtype: float64

In [66]:
# apply the exchange rate calculation to the entire column, round to two decimals
autos['price'] = round((autos['price'] / 1.11),2)

In [67]:
# check against our reference to verify the change
autos['price'][0:2] 

0    4504.50
1    7657.66
Name: price, dtype: float64

In [68]:
# rename the column for clarity
autos.rename({'price':'price_usd'},axis = 1, inplace=True)

In [69]:
# verify our name columns have changed
autos.columns

Index(['date_crawled', 'name', 'seller', 'price_usd', 'abtest', 'vehicle_type',
       'registration_year', 'gearbox', 'horse_power', 'model',
       'odometer_miles', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'count_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

All necessary changes to the __price__ column have been made.

### Examining the Name Column for Patterns

In [70]:
autos[['name','brand','model','vehicle_type']].head(10)

,name,brand,model,vehicle_type
0,Peugeot_807_160_NAVTECH_ON_BOARD,peugeot,andere,van
1,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,bmw,7er,sedan
2,Volkswagen_Golf_1.6_United,volkswagen,golf,sedan
3,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,smart,fortwo,compact
4,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,ford,focus,crossover
5,Chrysler_Grand_Voyager_2.8_CRD_Aut.Limited_Sto...,chrysler,voyager,van
6,VW_Golf_III_GT_Special_Electronic_Green_Metall...,volkswagen,golf,sedan
7,Golf_IV_1.9_TDI_90PS,volkswagen,golf,sedan
8,Seat_Arosa,seat,arosa,NaN
9,Renault_Megane_Scenic_1.6e_RT_Klimaanlage,renault,megane,van


There is a clear pattern that indicates a naming convention. The first two to three values followed by an underscore match the brand and model columns. What follows, however, is inconsistent. It appears to be feature descriptors for the car, but follows no immediately apparent pattern. Given that we alread have brand and model columns to work off of, and features are too granular for this analysis, we will not use these in our aggregate analysis.

# Observations of Data and Actions to Take (Update: 4)

The summary data tells us that there are a few areas where the data can be improved upon. Below are initial observations.

- ~~The columns ought to be renamed to follow snakecase convention and could be more descriptive.~~

- ~~Certain columns are in _object_ type and should be changed to either _int_ or _float_ type in preparation for analysis. Relevant columns include: __price__ and __odometer__.~~

- There are columns with _null_ values and these need to be analyzed in order to make descisions on how to handle them. i.e. do we leave these values alone, find and include the missing information, or completely remove the values/rows/columns. Target columns include: __vehicle_type__, __gearbox__, __model__, __fuel_type__, and __unrepaired_damage__.

- ~~The __name__ column has information encoded within it that may be worth pulling out into their own columns, such as the model and additional features of the vehicles. Exploration is needed to understand encoded contents.~~

- ~~There are severeal columns that are in German and should be translated for our English speaking audience. Relevant columns include: __seller__, __offer_type__, __vehicle_type__, __gearbox__, __fuel_type__, and __unrepaired_damage__. Similarly, decisions will be made where relevant to best translate for an American audience, such as changing from km to mile.~~

- ~~Several columns represent date values but are not in datetime format. Columns: __date_crawled__, __last_seen__, __ad_created__, __registration_month__, and __registration_year__~~


___

- ~~The __date_crawled__ and __last_seen__ columns may not provide useful information.~~

- The __count_picutures__ column does not appear to contain any useful information as all values are zero.

- ~~The __registration_year__ appears to have some dirty data, with a min of 1000 and max of 9999.~~

- ~~The __registration_month__ has a numerical month and has 0 for its minimum and 12 for its maximum. This would leave us with 13 months. We should explore whether there are some miscategorized zero values that should begin with 1. ~~

- ~~__power_ps__ is the European equivalent of horsepower. 1 PS or metric horsepower(EU) is equivalent to .989632 HP or mechanical horsepower(US). For reference 1,000 HP is attainable, but very rare (and expensive). The maximum we see in our __power_ps__ column of 17,700 is therefore unreasonable. This column should be invesitgated for any cars over 1,000 HP. The conversion formula we will use is HP = (PS / 1.104).~~

- ~~Check whether __odometer__ values are ranges, and if they are clean the data to make it more clear.~~

- ~~Several columns are only one of two values. Their value towards analysis should be considered. Relevant columns include: __seller__, __offer_type__, __abtest__, __gearbox__, and __unrepaired_damage__~~

___


- ~~Evaluate the extent of bad __pricing__ data and determine the best way to clean the series.~~

- Do certain car __brands__ or __models__ have increased __registrations_month__ than other car brands? (There is a loose link between registration and purchase)

### Dropping Unhelpful Columns

In [71]:
autos

,date_crawled,name,seller,price_usd,abtest,vehicle_type,registration_year,gearbox,horse_power,model,odometer_miles,registration_month,fuel_type,brand,unrepaired_damage,ad_created,count_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,private,4504.50,control,van,2004,manual,143.12,andere,93000,March,propane,peugeot,no,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,private,7657.66,control,sedan,1997,automatic,259.06,7er,93000,June,gasoline,bmw,no,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,private,8099.10,test,sedan,2009,manual,92.39,golf,44000,July,gasoline,volkswagen,no,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,private,3918.92,control,compact,2007,automatic,64.31,fortwo,44000,June,gasoline,smart,no,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,private,1216.22,test,crossover,2003,manual,NaN,focus,93000,July,gasoline,ford,no,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2016-03-27 14:38:19,Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon,private,22432.43,control,sedan,2011,automatic,216.49,q5,62000,January,diesel,audi,no,2016-03-27 00:00:00,0,82131,2016-04-01 13:47:40
49996,2016-03-28 10:50:25,Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...,private,1783.78,control,convertible,1996,manual,67.93,astra,93000,May,gasoline,opel,no,2016-03-28 00:00:00,0,44807,2016-04-02 14:18:02
49997,2016-04-02 14:44:48,Fiat_500_C_1.2_Dualogic_Lounge,private,11891.89,test,convertible,2014,automatic,62.50,500,3000,November,gasoline,fiat,no,2016-04-02 00:00:00,0,73430,2016-04-04 11:47:27
49998,2016-03-08 19:25:42,Audi_A3_2.0_TDI_Sportback_Ambition,private,20630.63,control,crossover,2013,manual,135.87,a3,25000,November,diesel,audi,no,2016-03-08 00:00:00,0,35683,2016-04-05 16:45:07


In [72]:
del autos['date_crawled']
del autos['abtest']
del autos['ad_created']
del autos['count_pictures']
del autos['postal_code']
del autos['last_seen']

In [73]:
autos

,name,seller,price_usd,vehicle_type,registration_year,gearbox,horse_power,model,odometer_miles,registration_month,fuel_type,brand,unrepaired_damage
0,Peugeot_807_160_NAVTECH_ON_BOARD,private,4504.50,van,2004,manual,143.12,andere,93000,March,propane,peugeot,no
1,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,private,7657.66,sedan,1997,automatic,259.06,7er,93000,June,gasoline,bmw,no
2,Volkswagen_Golf_1.6_United,private,8099.10,sedan,2009,manual,92.39,golf,44000,July,gasoline,volkswagen,no
3,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,private,3918.92,compact,2007,automatic,64.31,fortwo,44000,June,gasoline,smart,no
4,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,private,1216.22,crossover,2003,manual,NaN,focus,93000,July,gasoline,ford,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon,private,22432.43,sedan,2011,automatic,216.49,q5,62000,January,diesel,audi,no
49996,Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...,private,1783.78,convertible,1996,manual,67.93,astra,93000,May,gasoline,opel,no
49997,Fiat_500_C_1.2_Dualogic_Lounge,private,11891.89,convertible,2014,automatic,62.50,500,3000,November,gasoline,fiat,no
49998,Audi_A3_2.0_TDI_Sportback_Ambition,private,20630.63,crossover,2013,manual,135.87,a3,25000,November,diesel,audi,no


## Analysis

Below, the top 20 most frequently posted for resale car brands are assigned to an object. The data is displayed as percentages.

In [74]:
top_20_brands = (autos['brand'] # Create the brand series and assign to an object
                 .value_counts(normalize = True, dropna = False) # create a frequency table by percentage
                 .sort_values(ascending = False) # sort by values, highest to lowest
                 .head(20)) # show the top 20

print(top_20_brands)

volkswagen        0.211529
bmw               0.110243
opel              0.107308
mercedes_benz     0.096538
audi              0.086805
ford              0.069776
renault           0.047114
peugeot           0.029870
fiat              0.025705
seat              0.018259
skoda             0.016403
nissan            0.015345
mazda             0.015237
smart             0.014201
citroen           0.014050
toyota            0.012798
hyundai           0.010036
sonstige_autos    0.009259
volvo             0.009129
mini              0.008806
Name: brand, dtype: float64


In [75]:
# take series index values and assign them to a list
top_20_brands_ls = top_20_brands.index

print(top_20_brands_ls)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat', 'seat', 'skoda', 'nissan', 'mazda', 'smart',
       'citroen', 'toyota', 'hyundai', 'sonstige_autos', 'volvo', 'mini'],
      dtype='object')


In [76]:
#autos.loc[autos['brand'] == 'volvo', 'price_usd'].mean()

In [77]:
# empty dictionary we will use to save a brand mean price values
brand_mean_price = {} 

# loop through the df looking for top 20 brands - calculate the mean price and assign to the dic
for brand in top_20_brands_ls:
    b_price_mean = round(autos.loc[autos['brand'] == brand, 'price_usd'].mean(),2)
    brand_mean_price[brand] = b_price_mean
    
print(brand_mean_price)

{'volkswagen': 4892.99, 'bmw': 7547.59, 'opel': 2704.53, 'mercedes_benz': 7757.16, 'audi': 8451.1, 'ford': 3378.31, 'renault': 2247.52, 'peugeot': 2805.28, 'fiat': 2547.46, 'seat': 3994.07, 'skoda': 5780.95, 'nissan': 4285.28, 'mazda': 3720.52, 'smart': 3240.0, 'citroen': 3399.44, 'toyota': 4655.04, 'hyundai': 4864.55, 'sonstige_autos': 11559.73, 'volvo': 4498.39, 'mini': 9585.09}


The top 5 most frequently sold brands are all German car brands, which, considering the source of the dataset, is as we might expect. 

\\$2,000 to $8,000 USD appears to be the sweet spot for the used car price point range.

There are price point categories we can identify fitting low, medium, and high within the \\$2,000 - $8,000 range. Low being around \\$3,000, medium around \\$5,000 and high around \\$8,000. Of the top five brands, one falls in low, Opel, one falls in medium, Volkswagen, and three fall in high-end, BMW, Mercedez Benz, and Audi.

The process will be repeated to calculate the mean mileage for the top 20 brands.

In [78]:
# empty dictionary we will use to save a brand mean milage values
brand_mean_mileage = {}

# loop through the df looking for top 20 brands - calculate the mean milage and assign to dic
for brand in top_20_brands_ls:
    b_mean_odo = round(autos.loc[autos['brand'] == brand, 'odometer_miles'].mean(),2)
    brand_mean_mileage[brand] = b_mean_odo
    
print(brand_mean_mileage)

{'volkswagen': 79927.56, 'bmw': 82336.53, 'opel': 80321.8, 'mercedes_benz': 81378.94, 'audi': 80192.19, 'ford': 77221.47, 'renault': 79671.09, 'peugeot': 78921.97, 'fiat': 72684.3, 'seat': 75453.9, 'skoda': 68886.84, 'nissan': 73469.76, 'mazda': 77321.53, 'smart': 61995.44, 'citroen': 74296.47, 'toyota': 72021.92, 'hyundai': 66466.67, 'sonstige_autos': 56738.93, 'volvo': 85962.17, 'mini': 54899.51}


In [79]:
# turn price dictionary into series 
bmp_series = pd.Series(brand_mean_price)

# turn odometer dictionary into series
bmo_series = pd.Series(brand_mean_mileage)

#print(bmo_series)

In [80]:
# turn the mean price series into a dataframe and assign a column name
bpo = pd.DataFrame(bmp_series, columns = ['mean_price'])

In [81]:
# add the odometer series to the dataframe
bpo['mean_odometer_miles'] = bmo_series

In [82]:
bpo

,mean_price,mean_odometer_miles
volkswagen,4892.99,79927.56
bmw,7547.59,82336.53
opel,2704.53,80321.80
mercedes_benz,7757.16,81378.94
audi,8451.10,80192.19
ford,3378.31,77221.47
renault,2247.52,79671.09
peugeot,2805.28,78921.97
fiat,2547.46,72684.30
seat,3994.07,75453.90


There is a range at which people more frequently sell their cars, 70-80k miles on the odometer. 

There doesn't appear to be a notable odometer variance among brands.

___

In [83]:
autos

,name,seller,price_usd,vehicle_type,registration_year,gearbox,horse_power,model,odometer_miles,registration_month,fuel_type,brand,unrepaired_damage
0,Peugeot_807_160_NAVTECH_ON_BOARD,private,4504.50,van,2004,manual,143.12,andere,93000,March,propane,peugeot,no
1,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,private,7657.66,sedan,1997,automatic,259.06,7er,93000,June,gasoline,bmw,no
2,Volkswagen_Golf_1.6_United,private,8099.10,sedan,2009,manual,92.39,golf,44000,July,gasoline,volkswagen,no
3,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,private,3918.92,compact,2007,automatic,64.31,fortwo,44000,June,gasoline,smart,no
4,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,private,1216.22,crossover,2003,manual,NaN,focus,93000,July,gasoline,ford,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon,private,22432.43,sedan,2011,automatic,216.49,q5,62000,January,diesel,audi,no
49996,Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...,private,1783.78,convertible,1996,manual,67.93,astra,93000,May,gasoline,opel,no
49997,Fiat_500_C_1.2_Dualogic_Lounge,private,11891.89,convertible,2014,automatic,62.50,500,3000,November,gasoline,fiat,no
49998,Audi_A3_2.0_TDI_Sportback_Ambition,private,20630.63,crossover,2013,manual,135.87,a3,25000,November,diesel,audi,no


In [84]:
# sort by most common vehicle types
ordered_vt = autos['vehicle_type'].value_counts(normalize = True).sort_index(ascending = True)
print(ordered_vt)

compact        0.239989
convertible    0.068506
coupe          0.056011
crossover      0.203440
other          0.008749
sedan          0.286726
suv            0.044704
van            0.091875
Name: vehicle_type, dtype: float64


In [85]:
#create a list of vehicle types based off the index
vehicle_types_ls = ordered_vt.index
print(vehicle_types_ls)

Index(['compact', 'convertible', 'coupe', 'crossover', 'other', 'sedan', 'suv',
       'van'],
      dtype='object')


In [87]:
# empty dictionary we will save vehicle types and their mean price 
vt_mean_price = {}

for v_type in vehicle_types_ls:
    vt_mean_pr = round(autos.loc[autos['vehicle_type'] == v_type, 'price_usd'].mean(),2)
    vt_mean_price[v_type] = vt_mean_pr
    
print(vt_mean_price)

{'compact': 2492.3, 'convertible': 9659.55, 'coupe': 10583.5, 'crossover': 5239.5, 'other': 3972.52, 'sedan': 5382.33, 'suv': 12069.66, 'van': 6018.76}


In [88]:
# turn vt price into a series
vt_mean_price_series = pd.Series(vt_mean_price)

print(vt_mean_price_series)

compact         2492.30
convertible     9659.55
coupe          10583.50
crossover       5239.50
other           3972.52
sedan           5382.33
suv            12069.66
van             6018.76
dtype: float64


In [90]:
# empty dictionary we will save vehicle types and their mean odometer
vt_mean_odo = {}

for v_type in vehicle_types_ls:
    vt_mean_o = round(autos.loc[autos['vehicle_type'] == v_type, 'odometer_miles'].mean(),2)
    vt_mean_odo[v_type] = vt_mean_o
    
print(vt_mean_odo)

{'compact': 74153.15, 'convertible': 70046.02, 'coupe': 74675.37, 'crossover': 84378.73, 'other': 77791.12, 'sedan': 79510.04, 'suv': 70603.47, 'van': 80298.36}


In [91]:
# create a series from the mean odometer by vehicle type library
vt_mean_odo_series = pd.Series(vt_mean_odo)
print(vt_mean_odo_series)

compact        74153.15
convertible    70046.02
coupe          74675.37
crossover      84378.73
other          77791.12
sedan          79510.04
suv            70603.47
van            80298.36
dtype: float64


In [92]:
tpo = pd.DataFrame(vt_mean_price_series, columns = ['mean_price'])

In [93]:
tpo['mean_odometer'] = vt_mean_odo_series

In [94]:
tpo

,mean_price,mean_odometer
compact,2492.30,74153.15
convertible,9659.55,70046.02
coupe,10583.50,74675.37
crossover,5239.50,84378.73
other,3972.52,77791.12
sedan,5382.33,79510.04
suv,12069.66,70603.47
van,6018.76,80298.36


There isn't a notable pattern between vehicle type and mean odometer. 

Vehicle types can be classified into three price groups: high(\~\\$10,000), medium(\~\\$5,000) and low(\~\\$3,000). 

SUV, coupe, and convertible are high price points.  
Van, sedan, and crossover are medium price points.  
Compact and other are low price points.  